<p align="center">
<img src="https://github.com/jina-ai/dalle-flow/blob/main/.github/banner.svg?raw=true" alt="DALL·E Flow: A Human-in-the-Loop workflow for creating HD images from text" width="60%">
<br>
</p>


<b>A Human-in-the-Loop<sup><a href="https://en.wikipedia.org/wiki/Human-in-the-loop">?</a></sup> workflow for creating HD images from text</b>


<p align=center>
<a href="https://slack.jina.ai"><img src="https://img.shields.io/badge/Slack-2.8k-blueviolet?logo=slack&amp;logoColor=white&style=flat-square" alt="Open in Google Colab"></a>
</p>

[![GitHub Repo stars](https://img.shields.io/github/stars/jina-ai/dalle-flow?style=social)](https://github.com/jina-ai/dalle-flow)



🚧 If you find your request fails, it is possible the server is occasionally down for maintaince. Please give it a try in 5 minutes.


- ⚠️ 2022/5/3 **The number of images is restrict to 9 for DALL·E Mega, and 16 for GLID3-XL**
- ⚠️ 2022/5/2 **Due to the massive requests now, the server is super busy.** You can deploy your own server by following [the instruction here](https://github.com/jina-ai/dalle-flow#server).


Using client is super easy. The following steps are best run in Jupyter notebook or [Google Colab](https://colab.research.google.com/github/jina-ai/dalle-flow/blob/main/client.ipynb).  

The only dependency you will need is [DocArray](https://github.com/jina-ai/docarray).

In [ ]:
!pip install "docarray[common]>=0.13.5" jina

In [ ]:
We have provided a demo server for you to play:

In [ ]:
server_url = 'grpc://dalle-flow.jina.ai:51005'

### Step 1: Generate via DALL·E Mega

Now let's define the prompt:

In [ ]:
prompt = 'an oil painting of a humanoid robot playing chess in the style of Matisse'

Let's submit it to the server and visualize the results:

In [ ]:
%%time

from docarray import Document

da = Document(text=prompt).post(server_url, parameters={'num_images': 9}).matches
da.plot_image_sprites(fig_size=(10,10), show_index=True)

Here we generate ~~16~~ (2022/5/3: **limit to max. 9** for online demo due to massive traffic) candidates as defined in `num_images`, which takes about ~1 minutes. You can use a smaller value if it is too long for you. The results are sorted by [CLIP-as-service](https://github.com/jina-ai/clip-as-service), with index-`0` as the best candidate judged by CLIP. 

### Step 2: Select and refinement via GLID3 XL

Of course, you may think differently. Notice the number in the top-left corner? Select the one you like the most and get a better view:

In [ ]:
fav_id = 3

fav = da[fav_id]

fav.display()

Now let's submit the selected candidates to the server for diffusion.

In [ ]:
%%time

diffused = fav.post(f'{server_url}/diffuse', parameters={'skip_rate': 0.5}).matches

diffused.plot_image_sprites(fig_size=(10,10), show_index=True)

This will give ~~36~~ (2022/5/3: **limit to max. 16** for online demo due to massive traffic) images based on the given image. You may allow the model to improvise more by giving `skip_rate` a near-zero value, or a near-one value to force its closeness to the given image. The whole procedure takes about ~1 minutes.


### Step 3: Select and upscale via SwanIR

Select the image you like the most, and give it a closer look:


In [ ]:
dfav_id = 34

fav = diffused[dfav_id]

fav.display()

Finally, submit to the server for the last step: upscaling to 1024 x 1024px.

In [ ]:
%%time

fav = fav.post(f'{server_url}/upscale')
fav.display()

That's it! It is _the one_. If not satisfied, please repeat the procedure. Btw, DocArray is a powerful and easy-to-use data structure for unstructured data. It is super productive for data scientists who work in cross-/multi-modal domain. To learn more about DocArray, [please check out the docs](https://docs.jina.ai).